In [13]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor

In [14]:
# Membaca file Excel dengan banyak sheet
file_path = 'data/dataset_primer_2.xlsx' 
output_path = 'data/dataset_processed.xlsx'


In [15]:
import pandas as pd

# Fungsi untuk mengisi data kosong dengan model prediktif
def fill_missing_data_with_model(df):
    target_columns = [col for col in df.columns if 'supply' not in col]
    
    for col in target_columns:
        known_data = df[df[col].notna()]
        missing_data = df[df[col].isna()]

        if not known_data.empty and not missing_data.empty:
            # Pisahkan fitur (X) dan target (y)
            X_train = known_data.drop(columns=target_columns)
            y_train = known_data[col]

            model = RandomForestRegressor(n_estimators=100, random_state=42)
            model.fit(X_train, y_train)

            # Prediksi nilai untuk data yang hilang
            X_missing = missing_data.drop(columns=target_columns)
            predicted_values = model.predict(X_missing)

            df.loc[df[col].isna(), col] = predicted_values

    return df






In [16]:
# Fungsi untuk membaca dan memproses seluruh sheet dalam file Excel
def process_excel_file(file_path, output_path):
    xls = pd.ExcelFile(file_path)
    with pd.ExcelWriter(output_path) as writer:
        for sheet_name in xls.sheet_names:
            df = pd.read_excel(xls, sheet_name=sheet_name)
            df_filled = fill_missing_data_with_model(df)
            df_filled.to_excel(writer, sheet_name=sheet_name, index=False)

In [17]:
process_excel_file(file_path, output_path)